## Zeros of Bessel functions
Let $j_{\nu,m}$ denote the $m$th zero of the Bessel function $J_\nu(z)$.
Asymptotically it holds

$$\mathop{j_{\nu,m}}\nolimits\nolimits\sim a-\frac{\mu-1}{8a}
-\frac{4(\mu-1)(7\mu-31)}{3(8a)^{3}}-\frac{32(\mu-1)(83\mu^{2}-982\mu+3779
)}{15(8a)^{5}}-\frac{64(\mu-1)(6949\mu^{3}-1\;53855\mu^{2}+1585743\mu-6277
237)}{105(8a)^{7}}-\cdots$$

for $m\to\infty$ where $a=(m+\frac{1}{2}\nu-\frac{1}{4})\pi$ and $\mu=4\nu^2$, cf.
http://dlmf.nist.gov/10.21#vi. 

The following function computes this approximation for $j_{\nu,m}$:

In [1]:
function besselj_zero_approx(ν::Integer, m::Integer)
    μ = 4*ν^2
    a = (m+ν/2-1/4)*π
    return (a
      -(μ-1)/(8a) - (4*(μ-1)*(7μ-31))/(3*(8*a)^3)
      -(32*(μ-1)*(83*μ^2-982*μ+3779))/(15*(8*a)^5)
      -(64*(μ-1)*(6949*μ^3-153855*μ^2+1585743*μ-6277237))/(105*(8*a)^7)
    )
end

besselj_zero_approx (generic function with 1 method)

Starting from an approximation for a zero of $J_\nu(z)$ the following function
improves this approximation iteratively using Newton's method.
For the derivative of $J_\nu(z)$ needed by Newton's method we use 
$$\mathop{J_{\nu}'}\!\left(z\right)=\mathop{J_{
\nu-1}}\nolimits\!\left(z\right)-\frac{\nu}{z}\mathop{J_{\nu}}
\nolimits\!\left(z\right).$$

In [2]:
function besselj_zero_iter(ν::Integer, z::AbstractFloat)
    T = typeof(z)
    ɛ = eps(T)
    for i = 1:200
        J = besselj(ν, z)
        if abs(J) < 1000ɛ 
            break
        end        
        Jprime = besselj(ν-1, z) - ν*J/z
        z -= J/Jprime   
        if i==200
            println("200 iterations, res=",abs(J))
        end
    end
    return z
end

besselj_zero_iter (generic function with 1 method)

In [3]:
besselj_zero(ν, m, T=Float64) = besselj_zero_iter(ν, convert(T, besselj_zero_approx(ν, m)))

besselj_zero (generic function with 2 methods)

Let $j_{\nu,m}'$ denote the $m$th zero of the derivative $J_\nu'(z)$.
Asymptotically it holds

$$\mathop{j_{\nu,m}}\nolimits',\mathop{y_{\nu,m}}\nolimits'\sim b-\frac{\mu+%
3}{8b}-\frac{4(7\mu^{2}+82\mu-9)}{3(8b)^{3}}-\frac{32(83\mu^{3}+2075\mu^{2}-30%
39\mu+3537)}{15(8b)^{5}}-\frac{64(6949\mu^{4}+2\;96492\mu^{3}-12\;48002\mu^{2}%
+74\;14380\mu-58\;53627)}{105(8b)^{7}}-\cdots,$$

for $m\to\infty$ where $b=(m+\frac{1}{2}\nu-\frac{3}{4})\pi$ and $\mu=4\nu^2$, cf.
http://dlmf.nist.gov/10.21#vi.  

The following function computes this approximation for $j_{\nu,m}'$:

In [4]:
function besseljprime_zero_approx(ν::Integer, m::Integer)
    #if (ν==0 && m==1)
    #    return 0.0
    #end
    if ν==0
        m += 1
    end
    μ = 4*ν^2
    b = (m+ν/2-3/4)*π
    return (b
      -(μ+3)/(8b) - 4*(7*μ^2+82*μ-9)/(3*(8*b)^3)
      -32*(83*μ^3+2075*μ^2-3039*μ+3537)/(15*(8*b)^5)
      -64*(6949*μ^4+296492*μ^3-1248002*μ^2+7414380*μ-5853627)/(105*(8*b)^7)
    )
end

besseljprime_zero_approx (generic function with 1 method)

Starting from an approximation for a zero of $J_\nu'(z)$ the following function
improves this approximation iteratively using Newton's method.
For the second derivative $J_\nu''(z)$ needed by Newton's method we use 
$$J_{\nu}''(z)=-\frac{1}{z}J_{\nu}'(z)+\left(\frac{\nu^2}{z^2}-1\right)J_{\nu}(z),$$
where $J_{\nu}'(z)$ is given by
$$\mathop{J_{\nu}'}\!\left(z\right)=\mathop{J_{
\nu-1}}\nolimits\!\left(z\right)-\frac{\nu}{z}\mathop{J_{\nu}}
\nolimits\!\left(z\right).$$

In [5]:
function besseljprime_zero_iter(ν::Integer, z::AbstractFloat)    
    T = typeof(z)
    ɛ = eps(T)
    for i = 1:200
        J = besselj(ν, z)
        J1 = besselj(ν-1, z) - ν*J/z
        if abs(J1) < 1000ɛ 
            break
        end       
        J2 = -J1/z + ((ν/z)^2-1)*J
        z -= J1/J2   
        if i==200
            println("200 iterations, res=",abs(J1))
        end
    end
    return z
end

besseljprime_zero_iter (generic function with 1 method)

In [6]:
function besseljprime_zero(ν::Integer, m::Integer, T=Float64) 
    #if (ν==0 && m==1)
    #    return 0.0
    #end
    besseljprime_zero_iter(ν, convert(T, besseljprime_zero_approx(ν, m)))
end

besseljprime_zero (generic function with 2 methods)

##Computations using Bigfloat and Quadmath/Float128
We calculate the first 100 zeros of the first 101 Bessel functions.

In [7]:
mmax = 100;
νmax = 100;

First we use __BigFloat__ with precision 113 bits, exactly the precison of Flot128:

In [8]:
set_bigfloat_precision(113)

dummy = besselj_zero(0, 1, BigFloat) 
j_bf_time = @elapsed j_bf = BigFloat[besselj_zero(ν, m, BigFloat) for ν=0:νmax, m=1:mmax]
j_bf

101x100 Array{BigFloat,2}:
 2.40482555769577276862163187932645475      …  3.13374266077527844719690245101902241e+02
 3.83170597020751231561443588630816071         3.14943472837767162458065600245612127e+02
 5.13562230184068255630140169013776579         3.16509535868128429585533287236806371e+02
 6.3801618959239835062366146419427028          3.18072501419144132055026568868862209e+02
 7.58834243450380438506963000798561741         3.19632414630425705634404677271588722e+02
 8.77148381595995401912286713340956027      …  3.21189319567600315733922205996408600e+02
 9.93610952421768489469308912696519267         3.22743259257683369021774485609679029e+02
 1.10863700192450838457627644359299991e+01     3.24294275722967013886561795050315039e+02
 1.22250922640046551756128047691073987e+01     3.25842410013500062317371915749048547e+02
 1.33543004774353310664199248834919227e+01     3.27387702238230273055544191699100512e+02
 1.44755006865545412384516376554131522e+01  …  3.2893019159487574928997503529758247

Next the same using __Float128__:

In [9]:
using Quadmath
dummy = besselj_zero(0, 1, Float128) 
j_f128_time = @elapsed j_f128 = Float128[besselj_zero(ν, m, Float128) for ν=0:νmax, m=1:mmax]
j_f128

101x100 Array{Quadmath.Float128,2}:
 2.40482555769577276862163187932645475e+00  …  3.13374266077527844719690245101902241e+02
 3.83170597020751231561443588630816071e+00     3.14943472837767162458065600245612127e+02
 5.13562230184068255630140169013776579e+00     3.16509535868128429585533287236806371e+02
 6.38016189592398350623661464194270280e+00     3.18072501419144132055026568868862209e+02
 7.58834243450380438506963000798561741e+00     3.19632414630425705634404677271588722e+02
 8.77148381595995401912286713340956027e+00  …  3.21189319567600315733922205996408600e+02
 9.93610952421768489469308912696519267e+00     3.22743259257683369021774485609679029e+02
 1.10863700192450838457627644359299991e+01     3.24294275722967013886561795050315039e+02
 1.22250922640046551756128047691073987e+01     3.25842410013500062317371915749048547e+02
 1.33543004774353310664199248834919212e+01     3.27387702238230273055544191699100512e+02
 1.44755006865545412384516376554131522e+01  …  3.2893019159487574928997503

Compare the accuracy:

In [10]:
maximum((j_f128-j_bf)./j_bf), eps(BigFloat), eps(Float128)

(8.00142307563709763593351964634848400e-29,1.92592994438723585305597794258492732e-34,1.92592994438723585305597794258492732e-34)

Float128 is about 4 times faster than BigFloat:

In [11]:
j_bf_time/j_f128_time

4.569166586963479

Finally the same using __Float64__:

In [12]:
dummy = besselj_zero(0, 1, Float64)
j_f64_time = @elapsed j_f64 = Float64[besselj_zero(ν, m, Float64) for ν=0:νmax, m=1:mmax]
j_f64

101x100 Array{Float64,2}:
   2.40483    5.52008    8.65373  …  303.95   307.091  310.233  313.374
   3.83171    7.01559   10.1735      305.519  308.66   311.802  314.943
   5.13562    8.41724   11.6198      307.085  310.226  313.368  316.51 
   6.38016    9.76102   13.0152      308.647  311.789  314.931  318.073
   7.58834   11.0647    14.3725      310.207  313.349  316.491  319.632
   8.77148   12.3386    15.7002   …  311.763  314.905  318.047  321.189
   9.93611   13.5893    17.0038      313.317  316.459  319.601  322.743
  11.0864    14.8213    18.2876      314.867  318.01   321.152  324.294
  12.2251    16.0378    19.5545      316.415  319.557  322.7    325.842
  13.3543    17.2412    20.807       317.959  321.102  324.245  327.388
  14.4755    18.4335    22.047    …  319.501  322.644  325.787  328.93 
  15.5898    19.616     23.2759      321.04   324.183  327.327  330.47 
  16.6982    20.7899    24.4949      322.576  325.72   328.863  332.007
   ⋮                              ⋱   

In [13]:
maximum((j_f128-j_f64)./j_f128), eps(Float64)

(5.24593089942647473579027937366297393e-13,2.220446049250313e-16)

Float64 is about 50 times faster than Float128:

In [14]:
j_f128_time/j_f64_time

45.125891168245566

Next we do the same as above with the zeros of the derivatives of the Bessel functions.

In [15]:
dummy = besseljprime_zero(0, 1, BigFloat) 
j1_bf_time = @elapsed j1_bf = BigFloat[besseljprime_zero(ν, m, BigFloat) for ν=0:νmax, m=1:mmax]

6.430396875

In [16]:
dummy = besseljprime_zero(0, 1, Float128) 
j1_f128_time = @elapsed j1_f128 = Float128[besseljprime_zero(ν, m, Float128) for ν=0:νmax, m=1:mmax]

1.751415851

In [17]:
maximum((j1_f128-j1_bf)./j1_f128)

1.11116840414273680272377231386047123e-30

In [18]:
j1_bf_time/j1_f128_time

3.6715420106129892

In [19]:
dummy = besselj_zero(0, 1, Float64)
j1_f64_time = @elapsed j1_f64 = Float64[besseljprime_zero(ν, m, Float64) for ν=0:νmax, m=1:mmax]

0.040962442

In [20]:
j1_f128_time/j1_f64_time

42.75662693645071

In [ ]:
maximum((j1_f128-j1_f64)./j1_f128)